In [1]:
clear;clc;close all;

In [ ]:
%% Read tarining and training
if ~exist('MNIST_train_img.mat','file')
    images=loadMNISTImages('train-images.idx3-ubyte');
    lb=loadMNISTLabels('train-labels.idx1-ubyte');
    train_images=reshape(images,[1,1,size(images,1),size(images,2)]);
    train_labels=lb';
    save('MNIST_train_img.mat','train_images');
    save('MNIST_train_label.mat','train_labels');
    size(train_images)
end


if ~exist('MNIST_test_img.mat','file')
    images=loadMNISTImages('t10k-images.idx3-ubyte');
    lb=loadMNISTLabels('t10k-labels.idx1-ubyte');
    test_images=reshape(images,[1,1,size(images,1),size(images,2)]);
    test_labels=lb';
    save('MNIST_test_img.mat','test_images');
    save('MNIST_test_label.mat','test_labels');
    size(test_images)
end




ans =

           1           1         784       60000


ans =

           1           1         784       10000



In [3]:
clear;clc;
load('MNIST_train_img.mat');

## Designing the following network
![](NET1.png)
![](NET2.png)
![](NET_f.png)

In [4]:
h_size=[100,200]


h_size =

   100   200



In [5]:
trainX=train_images;
for i=1:size(h_size,2)
    s=0;
    [m,n,p,q]=size(trainX);
    if ~exist(strcat('network',num2str(i),'.mat'),'file')
        layers=[imageInputLayer([m,n,p])...
        fullyConnectedLayer(h_size(i),'Name','fc1')...
        fullyConnectedLayer(p)...
        regressionLayer]';
      s=1;  
    else
        load(strcat('network',num2str(i),'.mat'));
        layers=net_pop.Layers;
    end
    if s==1
        ep=2;%input('enter number of epoch :');
                options=trainingOptions('sgdm','MiniBatchSize',200,...
                'InitialLearnRate',0.01,...
                    'MaxEpochs',ep,'ExecutionEnvironment','gpu',...
                    'Shuffle','once','VerboseFrequency',10,'Verbose',true,...
                    'L2Regularization',...
                    0.001,'Plots','training-progress');
                [net_pop,v]=trainNetwork(trainX,trainX,layers,options);
                save(strcat('network',num2str(i),'.mat'),'net_pop','v');
    else
        a=1;%input('Do you want to train press 1 to yes:');
            if a==1
                ep=2;%input('enter number of epoch :')
                options=trainingOptions('sgdm','MiniBatchSize',200,...
                'InitialLearnRate',...
                0.0001,'MaxEpochs',ep,'ExecutionEnvironment','gpu',...
                    'VerboseFrequency',10,'Verbose',true,'L2Regularization',...
                    0.001,'Plots','training-progress');
                [net_pop,v]=trainNetwork(trainX,trainX,layers,options);
                delete(strcat('network',num2str(i),'.mat'))
                pause(0.001)
                save(strcat('network',num2str(i),'.mat'),'net_pop','v');
                pause(0.1)
            end
    end
    feat=activations(net_pop,trainX,'fc1');
    size(feat)
    trainX=feat;
end

Initializing image normalization.
|========================================================================================|
|  Epoch  |  Iteration  |  Time Elapsed  |  Mini-batch  |  Mini-batch  |  Base Learning  |
|         |             |   (hh:mm:ss)   |     RMSE     |     Loss     |      Rate       |
|========================================================================================|
|       1 |           1 |       00:00:03 |         9.42 |         44.4 |          0.0100 |
|       1 |          10 |       00:00:04 |         7.98 |         31.8 |          0.0100 |
|       1 |          20 |       00:00:04 |         6.34 |         20.1 |          0.0100 |
|       1 |          30 |       00:00:04 |         5.66 |         16.0 |          0.0100 |
|       1 |          40 |       00:00:04 |         5.05 |         12.7 |          0.0100 |
|       1 |          50 |       00:00:04 |         4.82 |         11.6 |          0.0100 |
|       1 |          60 |       00:00:05 |         4.45 

![](AE1.png)

![](AE2.png)

In [6]:
trainX=train_images;
s=0;
load('MNIST_train_label.mat');
u=unique(train_labels);
[m,n,p,q]=size(trainX);
for i=1:size(h_size,2)
    f(1,i)=fullyConnectedLayer(h_size(i),'Name',strcat('fc',num2str(i)));
end
if ~exist('f_network.mat','file')
    layers=[imageInputLayer([m,n,p])...
    f...
    fullyConnectedLayer(size(u,2))...
    softmaxLayer...
    classificationLayer]';
  s=1;  
else
    load('f_network.mat');
    layers=net_pop.Layers;
end
for i=1:size(h_size,2)
    load(strcat('network',num2str(i),'.mat'));
    l=net_pop.Layers;
    layers(i+1).Weights=l(2).Weights;
    layers(i+1).Bias=l(2).Bias;
end


for i=1:size(u,2)
    cu{i}=strcat('c',num2str(u(i)));
end
train_target=categorical(train_labels,u,cu);

if s==1
    ep=10;%input('enter number of epoch :');
            options=trainingOptions('sgdm','MiniBatchSize',200,...
            'InitialLearnRate',0.01,...
                'MaxEpochs',ep,'ExecutionEnvironment','gpu',...
                'Shuffle','once','VerboseFrequency',10,'Verbose',true,...
                'L2Regularization',...
                0.001,'Plots','training-progress');
            [net_pop,v]=trainNetwork(trainX,train_target,layers,options);
            save('f_network.mat','net_pop','v');
else
    a=1;%input('Do you want to train press 1 to yes:');
        if a==1
            ep=10;%input('enter number of epoch :')
            options=trainingOptions('sgdm','MiniBatchSize',200,...
            'InitialLearnRate',...
            0.0001,'MaxEpochs',ep,'ExecutionEnvironment','gpu',...
                'VerboseFrequency',10,'Verbose',true,'L2Regularization',...
                0.001,'Plots','training-progress');
            [net_pop,v]=trainNetwork(trainX,train_target,layers,options);
            delete('f_network.mat')
            pause(0.001)
            save('f_network.mat','net_pop','v');
            pause(0.1)
        end
end


Initializing image normalization.
|========================================================================================|
|  Epoch  |  Iteration  |  Time Elapsed  |  Mini-batch  |  Mini-batch  |  Base Learning  |
|         |             |   (hh:mm:ss)   |   Accuracy   |     Loss     |      Rate       |
|========================================================================================|
|       1 |           1 |       00:00:17 |       11.50% |       2.3062 |          0.0100 |
|       1 |          10 |       00:00:17 |       65.50% |       2.0603 |          0.0100 |
|       1 |          20 |       00:00:17 |       74.50% |       1.5099 |          0.0100 |
|       1 |          30 |       00:00:18 |       74.00% |       1.1181 |          0.0100 |
|       1 |          40 |       00:00:18 |       74.50% |       0.8305 |          0.0100 |
|       1 |          50 |       00:00:18 |       80.00% |       0.6930 |          0.0100 |
|       1 |          60 |       00:00:18 |       89.50% 

![](f_train.png)

In [8]:
%%testing 
clear;
disp('starting the testing phase ......')
load('f_network.mat')
load('MNIST_test_img.mat');
load('MNIST_test_label.mat');
[m,n,p,q]=size(test_images);
u=unique(test_labels);
for i=1:size(u,2)
    cu{i}=strcat('c',num2str(u(i)));
end
test_target=categorical(test_labels,u,cu);
p=predict(net_pop,test_images);
[v,p]=max(p');
acc=(sum(p-1==test_labels)/size(test_labels,2))*100;
disp(strcat('Accuracy is :',char({' '}),num2str(acc)))

starting the testing phase ......
Accuracy is :92.19
